# 🌊 DepthFlow API Server on Google Colab

在 Google Colab 上部署 DepthFlow REST API 服務，提供 2.5D 視差動畫生成功能。

## ✨ 優勢特點
- 🆓 **完全免費**：享受免費 GPU 加速（T4/P100/V100）
- 🌐 **公網訪問**：自動獲得公網 IP，手機直接連接
- ⚡ **快速部署**：5-10 分鐘完成部署，無需複雜配置
- 🔄 **免安裝**：瀏覽器即可操作，無需本地環境
- 📱 **移動友善**：完美適配手機 App 測試

## 📋 使用說明
1. **啟用 GPU**：Runtime → Change runtime type → Hardware accelerator → GPU
2. **依序執行**：從上到下執行每個程式碼儲存格
3. **複製 URL**：從第 8 個儲存格獲得公網 URL
4. **配置 App**：將 URL 設定到手機 App 中
5. **開始測試**：使用手機 App 拍照生成 2.5D 動畫

⚠️ **重要提醒**：請保持此頁面開啟，關閉後服務將停止運行

In [ ]:
# 🔧 步驟 1：環境檢查與 GPU 設定
import torch
import os

print("🔍 正在檢查運行環境...")
print(f"Python 版本: {os.popen('python --version').read().strip()}")
print(f"PyTorch 版本: {torch.__version__}")
print(f"CUDA 可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"✅ GPU 設備: {torch.cuda.get_device_name(0)}")
    print(f"📊 GPU 記憶體: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("🚀 GPU 加速已啟用，將提供更快的處理速度！")
else:
    print("⚠️  未檢測到 GPU，建議啟用以獲得更好性能：")
    print("   Runtime → Change runtime type → Hardware accelerator → GPU → Save")
    print("   然後重新啟動並執行此儲存格")

In [ ]:
# 📦 步驟 2：安裝必要套件（約 2-3 分鐘）
print("📦 開始安裝 DepthFlow API 所需的套件...")
print("⏳ 預計需要 2-3 分鐘，請耐心等待...")

# 核心 Web 框架
!pip install fastapi==0.115.14
!pip install uvicorn[standard]==0.30.0
!pip install python-multipart>=0.0.18

# 數據處理與模型
!pip install pydantic==2.11.7
!pip install pydantic-settings==2.5.0
!pip install depthflow==0.9.1

# 任務處理（簡化版，不啟用 Redis）
!pip install celery==5.3.6
!pip install redis==5.2.0

# 圖像處理
!pip install pillow==10.4.0
!pip install numpy>=1.26.0

# 文件處理與環境
!pip install aiofiles==24.1.0
!pip install python-dotenv==1.0.1

# 監控
!pip install prometheus-client==0.21.0

print("✅ 所有依賴套件安裝完成！")
print("🎯 接下來將設定網路隧道...")

In [ ]:
# 🌐 步驟 3：設定 ngrok 網路隧道
print("🌐 安裝 ngrok（用於建立公網隧道）...")

!pip install pyngrok

print("✅ ngrok 安裝完成！")
print("")
print("💡 專業提示：註冊 ngrok 帳號可獲得更穩定的服務")
print("📝 註冊步驟：")
print("   1. 訪問 https://ngrok.com/ 並註冊帳號")
print("   2. 取得您的 authtoken")
print("   3. 取消註解下面這行並填入您的 token")
print("   4. 重新執行此儲存格")
print("")
print("🔧 如果有 authtoken，請取消註解下面這行：")

# 註冊 ngrok（可選，但建議註冊以提升穩定性）
# 到 https://ngrok.com/ 註冊並取得 authtoken
# 然後取消註解下面這行並填入您的 token
# !ngrok authtoken YOUR_NGROK_TOKEN

print("🎯 如未註冊也可正常使用，但 URL 可能會較頻繁變動")

In [ ]:
# 📁 步驟 4：創建專案目錄結構
import os

print("📁 正在創建 DepthFlow API 專案結構...")

# 創建主要目錄
directories = [
    'depthflow_api/app/api',
    'depthflow_api/app/models', 
    'depthflow_api/app/services',
    'depthflow_api/app/tasks',
    'depthflow_api/app/utils',
    'depthflow_api/storage/uploads',
    'depthflow_api/storage/outputs'
]

for directory in directories:
    os.makedirs(directory, exist_ok=True)
    print(f"  ✓ {directory}")

print("")
print("✅ 專案目錄結構創建完成！")
print("📊 結構概覽：")
print("  📂 depthflow_api/")
print("    ├── 📂 app/          # 應用程式核心")
print("    └── 📂 storage/      # 文件存儲")
print("        ├── 📂 uploads/  # 上傳的圖片")
print("        └── 📂 outputs/  # 生成的動畫")

In [ ]:
# 📝 步驟 5：下載 DepthFlow API 源代碼並修復模組結構
import subprocess
import os
import shutil

print("📝 正在從 GitHub 下載 DepthFlow API 源代碼...")
print("🔗 倉庫：https://github.com/Chuanyin1202/depthflow-api")

# 確保在正確的工作目錄
os.chdir('/content')

try:
    # 使用 subprocess 正確捕獲錯誤
    print("⏳ 正在克隆倉庫...")
    result = subprocess.run([
        'git', 'clone', 
        'https://github.com/Chuanyin1202/depthflow-api.git', 
        'temp_api'
    ], capture_output=True, text=True, check=True)
    
    print("✅ Git 克隆成功")
    
    # 檢查下載的內容
    if os.path.exists('temp_api'):
        print("📋 下載內容檢查：")
        for item in os.listdir('temp_api'):
            print(f"  📄 {item}")
    
    print("⏳ 正在複製文件到工作目錄...")
    
    # 複製文件（使用更安全的方式）
    if os.path.exists('depthflow_api'):
        shutil.rmtree('depthflow_api')  # 清除舊目錄
    
    shutil.copytree('temp_api', 'depthflow_api')
    print("✅ 文件複製完成")
    
    # 清理臨時目錄
    shutil.rmtree('temp_api')
    print("🧹 臨時文件清理完成")
    
    # 檢查專案結構
    print("\n🔍 檢查專案結構：")
    for root, dirs, files in os.walk('/content/depthflow_api/app'):
        level = root.replace('/content/depthflow_api/app', '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")
    
    # 確保所有 Python 套件目錄都有 __init__.py
    print("\n🔧 檢查並創建 __init__.py 文件...")
    init_files = [
        '/content/depthflow_api/app/__init__.py',
        '/content/depthflow_api/app/models/__init__.py',
        '/content/depthflow_api/app/api/__init__.py',
        '/content/depthflow_api/app/services/__init__.py',
        '/content/depthflow_api/app/tasks/__init__.py',
        '/content/depthflow_api/app/utils/__init__.py'
    ]
    
    for init_file in init_files:
        dir_path = os.path.dirname(init_file)
        if os.path.exists(dir_path):  # 只有目錄存在才創建 __init__.py
            if not os.path.exists(init_file):
                with open(init_file, 'w') as f:
                    f.write('# This file makes Python treat the directory as a package\n')
                print(f"  ✅ 創建 {os.path.basename(os.path.dirname(init_file))}/__init__.py")
            else:
                print(f"  ✓ {os.path.basename(os.path.dirname(init_file))}/__init__.py 已存在")
        else:
            print(f"  ⚠️ 目錄不存在：{dir_path}")
    
    # 驗證關鍵文件
    print("\n📋 驗證關鍵組件：")
    critical_files = [
        '/content/depthflow_api/app/main.py',
        '/content/depthflow_api/app/api/routes.py', 
        '/content/depthflow_api/app/services/depthflow.py',
        '/content/depthflow_api/app/models/schemas.py',
        '/content/depthflow_api/app/config.py',
        '/content/depthflow_api/requirements.txt'
    ]
    
    missing_files = []
    for file in critical_files:
        if os.path.exists(file):
            print(f"  ✓ {os.path.relpath(file, '/content/depthflow_api')}")
        else:
            print(f"  ❌ {os.path.relpath(file, '/content/depthflow_api')} 缺失")
            missing_files.append(file)
    
    if missing_files:
        print(f"\n⚠️ 發現 {len(missing_files)} 個缺失文件")
        print("🔧 可能需要檢查 GitHub 倉庫是否包含所有必要文件")
    else:
        print("\n✅ 所有關鍵文件驗證通過")
        
    print("\n✅ 源代碼部署完成！")
    
except subprocess.CalledProcessError as e:
    print(f"❌ Git 操作失敗：{e}")
    print(f"錯誤詳情：{e.stderr}")
    
    # 提供具體的解決建議
    if "could not read Username" in str(e.stderr):
        print("\n💡 解決方案：")
        print("1. 倉庫可能是 private，請設為 public")
        print("2. 或者手動上傳代碼文件到 Colab")
        print("3. 到 GitHub → Settings → Change repository visibility → Public")
    elif "not found" in str(e.stderr):
        print("\n💡 倉庫不存在或網路問題，請檢查：")
        print("1. 倉庫 URL 是否正確")
        print("2. 網路連接是否正常")
    
    raise Exception("代碼下載失敗，請根據上述建議解決問題")
    
except Exception as e:
    print(f"❌ 下載過程出現意外錯誤：{e}")
    print("\n🔧 請嘗試：")
    print("1. 重新執行此儲存格")
    print("2. 檢查網路連接")
    print("3. 確認 GitHub 倉庫可正常訪問")
    raise

In [ ]:
# ⚙️ 步驟 6：Colab 環境配置（修正 Pydantic 格式）
import os
import json

print("⚙️ 正在創建針對 Colab 優化的環境配置...")

# 確保在正確的目錄
os.chdir('/content')

# 確保目錄存在
if not os.path.exists('depthflow_api'):
    print("❌ depthflow_api 目錄不存在，請先執行儲存格 4 和 5")
    raise FileNotFoundError("請先執行前面的儲存格創建目錄和下載代碼")

# 創建針對 Colab 優化的 .env 文件（修正 Pydantic JSON 格式）
env_content = '''# Colab Environment Settings
APP_NAME=DepthFlow API (Colab)
APP_VERSION=1.0.0-colab
DEBUG=false

# API Configuration - 使用 8081 避免端口衝突
API_HOST=0.0.0.0
API_PORT=8081
API_PREFIX=/api/v1

# File upload settings
MAX_UPLOAD_SIZE=20971520
ALLOWED_EXTENSIONS=["jpg","jpeg","png"]

# Storage paths (絕對路徑)
UPLOAD_PATH=/content/depthflow_api/storage/uploads
OUTPUT_PATH=/content/depthflow_api/storage/outputs

# Redis settings (簡化配置)
REDIS_HOST=localhost
REDIS_PORT=6379
REDIS_DB=0

# Celery settings (使用 Redis)
CELERY_BROKER_URL=redis://localhost:6379/0
CELERY_RESULT_BACKEND=redis://localhost:6379/0

# DepthFlow settings optimized for Colab
DEPTHFLOW_MAX_RESOLUTION=1920
DEPTHFLOW_DEFAULT_FPS=30
DEPTHFLOW_DEFAULT_DURATION=3

# Security (開發環境)
API_KEY_ENABLED=false
API_KEY=colab-dev-key

# CORS for mobile access (JSON 格式)
CORS_ORIGINS=["*"]
CORS_ALLOW_CREDENTIALS=true
CORS_ALLOW_METHODS=["*"]
CORS_ALLOW_HEADERS=["*"]
'''

try:
    # 寫入配置文件
    with open('/content/depthflow_api/.env', 'w') as f:
        f.write(env_content)
    
    print("✅ 環境配置文件創建成功！")
    
    # 驗證配置文件
    if os.path.exists('/content/depthflow_api/.env'):
        file_size = os.path.getsize('/content/depthflow_api/.env')
        print(f"📄 配置文件大小：{file_size} bytes")
    
    # 測試配置是否能正確加載（修正版本）
    print("🧪 測試配置加載...")
    
    # 先切換到正確的目錄
    original_dir = os.getcwd()
    os.chdir('/content/depthflow_api')
    
    test_script = '''
import sys
import os

# 確保在正確的目錄
print(f"當前工作目錄：{os.getcwd()}")
print(f".env 文件存在：{os.path.exists('.env')}")

# 清除可能的緩存
if 'app.config' in sys.modules:
    del sys.modules['app.config']
if 'app' in sys.modules:
    del sys.modules['app']

try:
    from app.config import Settings
    # 直接創建新實例，不使用緩存的 get_settings
    settings = Settings(_env_file='.env')
    print("✅ 配置加載成功")
    print(f"  🌐 API 端口：{settings.api_port}")
    print(f"  📁 上傳路徑：{settings.upload_path}")
    print(f"  📋 允許格式：{settings.allowed_extensions}")
    print(f"  🔒 CORS 來源：{settings.cors_origins}")
    
    # 驗證是否真的讀取了 .env
    if settings.api_port == 8081:
        print("✅ 成功從 .env 文件讀取配置")
    else:
        print("⚠️ 可能使用了默認值而非 .env 配置")
        
except Exception as e:
    print(f"❌ 配置加載失敗：{e}")
    import traceback
    traceback.print_exc()
'''
    
    exec(test_script)
    
    # 切回原目錄
    os.chdir(original_dir)
    
    print("\n📊 配置摘要：")
    print("  🌐 API 端口：8081（避免衝突）")
    print("  📁 存儲路徑：絕對路徑配置")
    print("  🔒 CORS：開放所有來源（移動端友善）")
    print("  ⚡ DepthFlow：Colab 優化參數")
    print("  📋 格式支援：JPG, JPEG, PNG")
    
except Exception as e:
    print(f"❌ 環境配置失敗：{e}")
    print("🔧 可能的解決方案：")
    print("1. 檢查 JSON 格式是否正確")
    print("2. 確認所有必要文件已下載")
    print("3. 重新執行儲存格 5 下載代碼")
    raise
finally:
    # 確保回到原目錄
    if 'original_dir' in locals():
        os.chdir(original_dir)

In [ ]:
# 🚀 步驟 7：啟動 DepthFlow API 服務（增強診斷）
import subprocess
import threading
import time
import os
import sys

print("🚀 正在啟動 DepthFlow API 服務...")

# 確保在正確的工作目錄
os.chdir('/content')

# 詳細驗證專案結構
print("🔍 詳細檢查專案結構...")

required_structure = {
    '/content/depthflow_api': '專案根目錄',
    '/content/depthflow_api/app': '應用程式目錄',
    '/content/depthflow_api/app/__init__.py': 'App 套件標識',
    '/content/depthflow_api/app/main.py': '主程式',
    '/content/depthflow_api/app/config.py': '配置文件',
    '/content/depthflow_api/app/models': 'Models 目錄',
    '/content/depthflow_api/app/models/__init__.py': 'Models 套件標識',
    '/content/depthflow_api/app/models/schemas.py': 'Schemas 定義',
    '/content/depthflow_api/app/api': 'API 目錄',
    '/content/depthflow_api/app/api/__init__.py': 'API 套件標識',
    '/content/depthflow_api/app/api/routes.py': 'API 路由',
    '/content/depthflow_api/.env': '環境配置'
}

missing_items = []
for path, description in required_structure.items():
    if os.path.exists(path):
        print(f"  ✅ {description}")
    else:
        print(f"  ❌ {description} 缺失：{path}")
        missing_items.append((path, description))

if missing_items:
    print(f"\n❌ 發現 {len(missing_items)} 個缺失項目")
    print("🔧 請重新執行儲存格 5 下載完整代碼")
    for path, desc in missing_items:
        print(f"  缺失：{desc}")
    raise FileNotFoundError("專案結構不完整")

print("✅ 專案結構驗證通過")

# 測試模組導入
print("\n🧪 測試 Python 模組導入...")
sys.path.insert(0, '/content/depthflow_api')

try:
    print("  測試配置模組...")
    from app.config import Settings
    print("  ✅ app.config 導入成功")
    
    print("  測試模型模組...")
    from app.models import schemas
    print("  ✅ app.models.schemas 導入成功")
    
    print("  測試 API 模組...")
    from app.api import routes
    print("  ✅ app.api.routes 導入成功")
    
    print("✅ 所有關鍵模組導入測試通過")
    
except ImportError as e:
    print(f"❌ 模組導入失敗：{e}")
    print("🔧 這可能是由於：")
    print("  1. 缺少 __init__.py 文件")
    print("  2. 代碼中有語法錯誤")
    print("  3. 依賴套件未正確安裝")
    raise

def start_server():
    """啟動服務的函數"""
    try:
        # 切換到專案目錄（使用絕對路徑）
        os.chdir('/content/depthflow_api')
        print(f"📁 服務工作目錄：{os.getcwd()}")
        
        # 啟動 FastAPI 服務
        print("🚀 正在啟動 FastAPI 服務...")
        result = subprocess.run([
            'python', '-m', 'app.main'
        ], capture_output=True, text=True)
        
        # 輸出服務日誌
        if result.stdout:
            print("📋 服務標準輸出：")
            print(result.stdout)
        if result.stderr:
            print("❌ 服務錯誤輸出：")
            print(result.stderr)
        if result.returncode != 0:
            print(f"❌ 服務異常結束，返回碼：{result.returncode}")
            
    except Exception as e:
        print(f"❌ 服務啟動異常：{e}")

# 在後台線程啟動服務
print("\n⚡ 在後台啟動服務...")
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

print("🕐 等待服務初始化...")
print("📝 注意：DepthFlow 首次啟動需要下載 AI 模型")
print("⏱️ 預計需要 1-3 分鐘，請耐心等待...")

# 智能等待服務啟動
max_wait_time = 60  # 增加到 60 秒
service_started = False

for i in range(max_wait_time):
    time.sleep(1)
    try:
        import requests
        response = requests.get("http://localhost:8081/api/v1/status", timeout=3)
        if response.status_code == 200:
            print(f"\n✅ 服務在第 {i+1} 秒成功啟動！")
            print(f"📊 服務狀態碼：{response.status_code}")
            data = response.json()
            print(f"🎯 服務資訊：{data.get('message', 'API 運行中')}")
            service_started = True
            break
    except:
        # 每 10 秒輸出一次進度
        if i % 10 == 0 and i > 0:
            print(f"⏳ 等待中... ({i}/{max_wait_time} 秒)")
        continue

if not service_started:
    print(f"\n⚠️ 服務在 {max_wait_time} 秒內未響應")
    print("🔧 可能的情況：")
    print("  1. 服務仍在下載模型，請稍後再試")
    print("  2. 服務啟動遇到問題，檢查上方錯誤訊息")
    print("  3. 網路較慢，模型下載需要更長時間")
    print("\n💡 建議：繼續執行下一步，如果隧道建立失敗再回來檢查")
else:
    print("🎯 服務啟動成功，準備建立公網隧道...")

print("\n🌐 準備建立 ngrok 隧道...")

In [ ]:
# 🌐 步驟 8：建立 ngrok 公網隧道（端口 8081）
from pyngrok import ngrok
import requests
import json
import time

print("🌐 正在建立 ngrok 公網隧道...")

try:
    # 建立隧道到 8081 端口
    public_tunnel = ngrok.connect(8081)
    public_url = public_tunnel.public_url
    
    print(f"🎉 ngrok 隧道建立成功！")
    print(f"🔗 公網 URL：{public_url}")
    
    # 等待一下讓隧道穩定
    print("⏳ 等待隧道穩定...")
    time.sleep(3)
    
    # 測試服務連接
    print("🧪 正在測試 API 服務...")
    
    test_endpoints = [
        "/api/v1/status",
        "/health", 
        "/docs"
    ]
    
    service_working = False
    
    for endpoint in test_endpoints:
        test_url = f"{public_url}{endpoint}"
        try:
            response = requests.get(test_url, timeout=10)
            if response.status_code == 200:
                print(f"✅ {endpoint} 響應正常 ({response.status_code})")
                service_working = True
                break
            else:
                print(f"⚠️ {endpoint} 響應異常 ({response.status_code})")
        except Exception as e:
            print(f"❌ {endpoint} 連接失敗：{e}")
    
    if service_working:
        print(f"\n🎉 DepthFlow API 服務已成功部署！")
        print(f"")
        print(f"📱 手機 App 配置：")
        print(f"API 端點: {public_url}")
        print(f"")
        print(f"🌐 快速連結：")
        print(f"📖 API 文檔: {public_url}/docs")
        print(f"📊 系統狀態: {public_url}/api/v1/status")
        print(f"❤️ 健康檢查: {public_url}/health")
        print(f"")
        print(f"📋 Flutter App 配置步驟：")
        print(f"1. 複製上方的 API 端點 URL")
        print(f"2. 修改 lib/services/api_service.dart 中的 defaultBaseUrl")
        print(f"3. 重新編譯並安裝 app：flutter build apk")
        print(f"4. 開始拍照測試！")
        print(f"")
        print(f"⚠️ 重要提醒：")
        print(f"- 此 URL 在 Colab 會話結束後失效")
        print(f"- 免費版 Colab 會話限制 12 小時")
        print(f"- 請保持此頁面開啟以維持服務運行")
        
    else:
        print(f"\n⚠️ 隧道建立成功，但 API 服務可能還在啟動中")
        print(f"🔗 隧道 URL：{public_url}")
        print(f"")
        print(f"🔧 請稍等 1-2 分鐘後手動測試：")
        print(f"📖 API 文檔: {public_url}/docs")
        print(f"📊 系統狀態: {public_url}/api/v1/status")
        print(f"")
        print(f"💡 如果仍無法訪問，請重新執行儲存格 7 啟動服務")
        
except Exception as e:
    print(f"❌ ngrok 隧道建立失敗：{e}")
    print(f"")
    print(f"🔧 可能的解決方案：")
    print(f"1. 重新執行此儲存格")
    print(f"2. 檢查網路連接")
    print(f"3. 如果問題持續，嘗試重啟 Colab runtime")
    print(f"4. 或者註冊 ngrok 帳號以獲得更穩定的服務")
    raise

In [ ]:
# 📊 步驟 9：服務監控與診斷
import time
import requests
import psutil
import subprocess
from datetime import datetime

def check_system_resources():
    """檢查系統資源使用情況"""
    print("💻 系統資源狀況：")
    print(f"  CPU 使用率：{psutil.cpu_percent()}%")
    print(f"  記憶體使用：{psutil.virtual_memory().percent:.1f}%")
    print(f"  可用記憶體：{psutil.virtual_memory().available / 1024**3:.1f} GB")
    print(f"  磁碟使用：{psutil.disk_usage('/').percent:.1f}%")

def check_service_status():
    """檢查服務狀態"""
    try:
        response = requests.get(f"{public_url}/api/v1/status", timeout=5)
        if response.status_code == 200:
            data = response.json()
            gpu_status = "✅ 可用" if data.get('gpu_available', False) else "❌ 不可用"
            print(f"🟢 API 服務正常運行")
            print(f"  📊 GPU 狀態：{gpu_status}")
            print(f"  ⚡ CPU 使用：{data.get('cpu_percent', 0):.1f}%")
            print(f"  🧠 記憶體使用：{data.get('memory_percent', 0):.1f}%")
            return True
        else:
            print(f"🟡 API 服務響應異常：{response.status_code}")
            return False
    except Exception as e:
        print(f"🔴 API 服務連接失敗：{e}")
        return False

def check_processes():
    """檢查相關進程"""
    print("🔍 進程檢查：")
    try:
        result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
        lines = result.stdout.split('\n')
        
        # 查找 Python API 進程
        api_processes = [line for line in lines if 'python' in line and 'app.main' in line]
        if api_processes:
            print(f"  ✅ 發現 {len(api_processes)} 個 API 進程")
        else:
            print("  ❌ 未發現 API 進程")
            
        # 查找 ngrok 進程
        ngrok_processes = [line for line in lines if 'ngrok' in line]
        if ngrok_processes:
            print(f"  ✅ 發現 {len(ngrok_processes)} 個 ngrok 進程")
        else:
            print("  ❌ 未發現 ngrok 進程")
            
    except Exception as e:
        print(f"  ❌ 進程檢查失敗：{e}")

def monitor_service():
    """執行完整的服務監控"""
    timestamp = datetime.now().strftime('%H:%M:%S')
    print(f"🕐 [{timestamp}] 服務監控報告")
    print("=" * 50)
    
    # 檢查公網 URL 是否可用
    if 'public_url' in globals():
        print(f"🌐 公網 URL：{public_url}")
        service_ok = check_service_status()
    else:
        print("❌ 公網 URL 未設定，請先執行儲存格 8")
        service_ok = False
    
    print()
    check_system_resources()
    print()
    check_processes()
    
    print("\n" + "=" * 50)
    if service_ok:
        print("✅ 整體狀態：正常運行")
        print("📱 可以開始使用手機 App 進行測試")
    else:
        print("⚠️ 整體狀態：需要檢查")
        print("🔧 建議重新執行儲存格 7 和 8")

# 執行監控
print("📊 正在執行服務監控...")
monitor_service()

print(f"\n💡 提示：可以重複執行此儲存格來查看即時狀態")
print(f"🔄 建議每隔 10-15 分鐘檢查一次服務狀態")

In [ ]:
# 🧪 步驟 10：測試界面與快速連結
from IPython.display import HTML
import os

# 檢查是否有公網 URL
if 'public_url' not in globals():
    print("❌ 公網 URL 未設定，請先執行儲存格 8 建立隧道")
else:
    # 創建互動式測試界面
    test_html = f'''
    <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                padding: 25px; border-radius: 15px; font-family: Arial, sans-serif; 
                color: white; margin: 10px 0;">
        <h2 style="margin-top: 0; text-align: center;">🌊 DepthFlow API 已就緒</h2>
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px; margin: 15px 0;">
            <h3>📱 手機 App 配置</h3>
            <p><strong>API 端點：</strong></p>
            <code style="background: rgba(0,0,0,0.3); padding: 8px; border-radius: 5px; 
                        display: block; margin: 5px 0; word-break: break-all;">{public_url}</code>
            
            <h4>🔧 配置步驟：</h4>
            <ol style="line-height: 1.6;">
                <li>複製上方的 API 端點 URL</li>
                <li>編輯 <code>mobile-app/lib/services/api_service.dart</code></li>
                <li>修改 <code>defaultBaseUrl</code> 為上方 URL</li>
                <li>執行 <code>flutter build apk</code> 重新編譯</li>
                <li>安裝到手機並開始測試！</li>
            </ol>
        </div>
        
        <div style="text-align: center; margin: 20px 0;">
            <a href="{public_url}/docs" target="_blank" 
               style="background: #28a745; color: white; padding: 12px 20px; text-decoration: none; 
                      border-radius: 8px; margin: 5px; display: inline-block; font-weight: bold;">
               📖 API 文檔
            </a>
            
            <a href="{public_url}/api/v1/status" target="_blank" 
               style="background: #007bff; color: white; padding: 12px 20px; text-decoration: none; 
                      border-radius: 8px; margin: 5px; display: inline-block; font-weight: bold;">
               📊 系統狀態
            </a>
            
            <a href="{public_url}/health" target="_blank" 
               style="background: #ffc107; color: black; padding: 12px 20px; text-decoration: none; 
                      border-radius: 8px; margin: 5px; display: inline-block; font-weight: bold;">
               ❤️ 健康檢查
            </a>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 15px; border-radius: 10px;">
            <h3>⚠️ 重要提醒</h3>
            <ul style="line-height: 1.6;">
                <li><strong>會話限制：</strong>免費版 Colab 最多 12 小時</li>
                <li><strong>保持開啟：</strong>關閉此頁面後服務將停止</li>
                <li><strong>URL 效期：</strong>重啟後會獲得新的 URL</li>
                <li><strong>處理時間：</strong>首次使用可能需要 1-2 分鐘下載模型</li>
            </ul>
        </div>
        
        <div style="background: rgba(46, 204, 113, 0.2); padding: 15px; border-radius: 10px; margin-top: 15px;">
            <h3>📈 效能提示</h3>
            <ul style="line-height: 1.6;">
                <li>🖼️ 建議圖片解析度：1920px 以下</li>
                <li>📦 支援格式：JPG, PNG（最大 20MB）</li>
                <li>⚡ GPU 加速：處理時間約 10-30 秒</li>
                <li>🎬 輸出格式：MP4 動畫影片</li>
            </ul>
        </div>
    </div>
    '''
    
    # 顯示測試界面
    display(HTML(test_html))
    
    # 提供純文字版本（方便複製）
    print("📋 純文字版本（方便複製）：")
    print("=" * 60)
    print(f"API 端點：{public_url}")
    print(f"API 文檔：{public_url}/docs")
    print(f"系統狀態：{public_url}/api/v1/status")
    print(f"健康檢查：{public_url}/health")
    print("=" * 60)
    
    # 檢查關鍵文件是否存在
    print("\n🔍 最終檢查：")
    critical_paths = [
        '/content/depthflow_api/app/main.py',
        '/content/depthflow_api/.env',
        '/content/depthflow_api/storage/uploads',
        '/content/depthflow_api/storage/outputs'
    ]
    
    all_good = True
    for path in critical_paths:
        if os.path.exists(path):
            print(f"✅ {path}")
        else:
            print(f"❌ {path} 缺失")
            all_good = False
    
    if all_good:
        print("\n🌟 所有組件就緒，可以開始使用手機 App 測試！")
    else:
        print("\n⚠️ 部分組件缺失，建議重新執行相關儲存格")

In [ ]:
# 💾 步驟 11：會話保持與服務監控
import time
import threading
import requests
from datetime import datetime

def keep_alive_and_monitor():
    """保持會話活躍並監控服務狀態"""
    counter = 0
    while True:
        try:
            time.sleep(300)  # 每 5 分鐘執行一次
            counter += 1
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            print(f"💓 Keep-alive #{counter} - {current_time}")
            
            # 檢查服務狀態
            if 'public_url' in globals():
                try:
                    response = requests.get(f"{public_url}/api/v1/status", timeout=10)
                    if response.status_code == 200:
                        data = response.json()
                        gpu_status = "🟢" if data.get('gpu_available', False) else "🔴"
                        print(f"  📊 服務狀態：正常 {gpu_status}")
                        print(f"  ⚡ 系統負載：CPU {data.get('cpu_percent', 0):.1f}% | 記憶體 {data.get('memory_percent', 0):.1f}%")
                    else:
                        print(f"  ⚠️ 服務響應異常：{response.status_code}")
                except Exception as e:
                    print(f"  ❌ 服務檢查失敗：{e}")
            else:
                print("  ❓ 公網 URL 未設定")
            
            # 每小時提醒一次
            if counter % 12 == 0:  # 12 * 5分鐘 = 1小時
                print(f"  ⏰ 已運行 {counter // 12} 小時")
                print(f"  💡 Colab 免費版限制 12 小時，請注意時間")
                
            # 每30分鐘檢查一次資源
            if counter % 6 == 0:   # 6 * 5分鐘 = 30分鐘
                import psutil
                memory_percent = psutil.virtual_memory().percent
                if memory_percent > 80:
                    print(f"  ⚠️ 記憶體使用率較高：{memory_percent:.1f}%")
                    print(f"  💡 建議：如遇問題可重啟 Runtime")
                    
        except KeyboardInterrupt:
            print("  ⏹️ 監控停止")
            break
        except Exception as e:
            print(f"  ❌ 監控過程出錯：{e}")

# 啟動保活和監控線程
print("💓 正在啟動會話保持機制...")
print("🔍 同時啟動服務監控功能...")

keep_alive_thread = threading.Thread(target=keep_alive_and_monitor)
keep_alive_thread.daemon = True
keep_alive_thread.start()

print("✅ Keep-alive 機制已啟動")
print("📊 服務監控已啟動")
print("")
print("🌟 DepthFlow API 服務現在正在運行中！")
print("")
print("📱 您現在可以：")
print("  1. 配置手機 App 的 API 端點")
print("  2. 重新編譯並安裝 App") 
print("  3. 開始拍照測試 2.5D 動畫生成")
print("")
print("💡 服務監控每 5 分鐘會輸出一次狀態")
print("🔄 您也可以隨時重新執行儲存格 9 來手動檢查狀態")
print("")
print("⚠️ 重要：請保持此 Colab 頁面開啟")
print("   關閉頁面會導致服務停止運行")